In [192]:
from river import drift
import numpy as np

from cmab.scm.domain.binary import BinaryDomain
from cmab.scm.distribution.bernoulli import Bernoulli
from cmab.scm.mechanism.custom import CustomMechanism
from cmab.scm.mechanism.xor import XORMechanism
from cmab.scm.scm import SCM
from cmab.environments import CausalBanditEnv, NSCausalBanditEnv
from cmab.environments.ns.scheduling.controlled_schedule import ControlledSchedule

In [193]:
SEED=42

### Data streams

In [194]:
rng = np.random.default_rng(seed=SEED)

# Simulating the data stream of X and Z
data_stream_X = np.concatenate([rng.binomial(1, 0.9, size=200), rng.binomial(1, 0.6, size=800)])
data_stream_Z = np.concatenate([rng.binomial(1, 0.75, size=400), rng.binomial(1, 0.45, size=200), rng.binomial(1, 0.8, size=400)])

In [195]:
V = ['X', 'Z', 'Y']
U = ['U_X', 'U_Z', 'U_Y']

domains = {
    'X': BinaryDomain(),
    'Z': BinaryDomain(),
    'Y': BinaryDomain()
}

P_X = Bernoulli(p=0.9)
P_Z = Bernoulli(p=0.75)
P_Y = Bernoulli(p=0.2)

mechanism_X = CustomMechanism(v_parents=[], u_parents=['U_X'], f=lambda _, u: u['U_X'])
mechanism_Z = CustomMechanism(v_parents=[], u_parents=['U_Z'], f=lambda _, u: u['U_Z'])
mechanism_Y = XORMechanism(v_parents=['X', 'Z'], u_parents=['U_Y'])

scm = SCM(
    U=U,
    V=V,
    domains=domains,
    P_u_marginals={
        'U_X': P_X,
        'U_Z': P_Z,
        'U_Y': P_Y
    },
    F={
        'X': mechanism_X,
        'Z': mechanism_Z,
        'Y': mechanism_Y
    },
    seed=SEED
)

reward_node = 'Y'
schedule = ControlledSchedule(exogenous=['U_X', 'U_Z', 'U_Z', 'U_Y'], new_params=[0.6, 0.45, 0.8, 0.5], every=200)
env = NSCausalBanditEnv(scm=scm, reward_node=reward_node, seed=SEED, atomic=True, shift_schedule=schedule)
print(f"Number of actions: {len(env.action_space)}")
print(f"Action space: {env.action_space}")

reward_stream = []
T = 1000

for i in range(T):
    if i % 100 == 0:
        print(f"Current parameters at step {i}: {[(k, v.p) for k, v in env.scm.P_u_marginals.items()]}")
    action = env.action_space[0]
    _, obs, _, _, _ = env.step(action)
    reward = obs['Y']
    reward_stream.append(reward)

Number of actions: 5
Action space: [frozenset(), frozenset({('X', 0)}), frozenset({('X', 1)}), frozenset({('Z', 0)}), frozenset({('Z', 1)})]
Current parameters at step 0: [('U_X', 0.9), ('U_Z', 0.75), ('U_Y', 0.2)]
Current parameters at step 100: [('U_X', 0.9), ('U_Z', 0.75), ('U_Y', 0.2)]
Current parameters at step 200: [('U_X', 0.9), ('U_Z', 0.75), ('U_Y', 0.2)]
Current parameters at step 300: [('U_X', 0.6), ('U_Z', 0.75), ('U_Y', 0.2)]
Current parameters at step 400: [('U_X', 0.6), ('U_Z', 0.75), ('U_Y', 0.2)]
Current parameters at step 500: [('U_X', 0.6), ('U_Z', 0.45), ('U_Y', 0.2)]
Current parameters at step 600: [('U_X', 0.6), ('U_Z', 0.45), ('U_Y', 0.2)]
Current parameters at step 700: [('U_X', 0.6), ('U_Z', 0.8), ('U_Y', 0.2)]
Current parameters at step 800: [('U_X', 0.6), ('U_Z', 0.8), ('U_Y', 0.2)]
Current parameters at step 900: [('U_X', 0.6), ('U_Z', 0.8), ('U_Y', 0.5)]


### Evaluate PH Detector

In [196]:
delta = 0.005
threshold = 20.0
min_instances = 30

In [197]:
ph = drift.PageHinkley(delta=delta, threshold=threshold, min_instances=min_instances)

for i, val in enumerate(data_stream_X):
    ph.update(val)
    if ph.drift_detected:
        print(f"PH change detected at index {i}, input value: {val}")

PH change detected at index 251, input value: 0


In [198]:
ph = drift.PageHinkley(delta=delta, threshold=threshold, min_instances=min_instances)

for i, val in enumerate(data_stream_Z):
    ph.update(val)
    if ph.drift_detected:
        print(f"PH change detected at index {i}, input value: {val}")

PH change detected at index 443, input value: 0
PH change detected at index 647, input value: 1


In [ ]:
delta = 0.05
threshold = 20.0
min_instances = 30

In [199]:
ph = drift.PageHinkley(delta=delta, threshold=threshold, min_instances=min_instances)

for i, val in enumerate(reward_stream):
    ph.update(val)
    if ph.drift_detected:
        print(f"PH change detected at index {i}, input value: {val}")

PH change detected at index 460, input value: 1


### Evaluate ADWIN Detector

In [200]:
delta = 0.002
clock = 32
max_buckets = 5
min_windw_length = 5
grace_period = 10

In [201]:
adwin = drift.ADWIN(delta=delta, clock=clock, max_buckets=max_buckets, min_window_length=min_windw_length, grace_period=grace_period)

for i, val in enumerate(data_stream_X):
    adwin.update(val)
    if adwin.drift_detected:
        print(f"ADWIN change detected at index {i}, input value: {val}")

ADWIN change detected at index 351, input value: 1


In [202]:
adwin = drift.ADWIN(delta=delta, clock=clock, max_buckets=max_buckets, min_window_length=min_windw_length, grace_period=grace_period)

for i, val in enumerate(data_stream_Z):
    adwin.update(val)
    if adwin.drift_detected:
        print(f"ADWIN change detected at index {i}, input value: {val}")

ADWIN change detected at index 479, input value: 0
ADWIN change detected at index 799, input value: 1


In [203]:
adwin = drift.ADWIN(delta=delta, clock=clock, max_buckets=max_buckets, min_window_length=min_windw_length, grace_period=grace_period)

for i, val in enumerate(reward_stream):
    adwin.update(val)
    if adwin.drift_detected:
        print(f"ADWIN change detected at index {i}, input value: {val}")

### Evaluate BOCPD

In [204]:
from cmab.algorithms.cpd.bodc import updateForecasterDistribution, updateForecasterDistribution_m, updateLaplacePrediction

In [205]:
Horizon = T
gamma = 1 / Horizon  # hazard
alphas = np.array([1])
betas = np.array([1])
ForecasterDistribution = np.array([1])
ChangePointEstimation = np.array([])

In [206]:
for i, val in enumerate(data_stream_X):
    EstimatedBestExpert = np.argmax(ForecasterDistribution) #Change-point estimation
    ChangePointEstimation = np.append(ChangePointEstimation,EstimatedBestExpert+1)
    ForecasterDistribution = updateForecasterDistribution(ForecasterDistribution, alphas, betas, val, gamma)
    (alphas, betas) = updateLaplacePrediction(alphas, betas, val) #Update the laplace predictor

print("Change Point Estimation:", len(ChangePointEstimation))

Change Point Estimation: 1000
